# FISH spot distance determination

this is a Jupyter notebook for processing of FISH images to detect spots in two channels and determining distances between them

the pipeline is made of *cells* of code - to execute a cell, just click into the cell and press *Ctrl-Enter*

In [ ]:
from util.detection_util import read_image_stack, distance_mat
from util.display_util import get_rgb_projected

from scipy.ndimage import gaussian_laplace, maximum_filter, maximum_position, zoom
from scipy.optimize import linear_sum_assignment
from scipy.signal import resample
from skimage.feature import peak_local_max
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import os

%matplotlib inline
plt.rcParams["figure.figsize"] = [10, 10]

# load images

set the folder that contains your images
we assume that the images in the folder are named as follows:
* Orange: o_[folder name].tif
* Red: r_[folder name].tif
* Blue: b_[folder name].tif

In [ ]:
folder = '/Volumes/tmp/Daaaaaavid/A557-13/'

channel1_file = os.path.join(folder, 'o_' + folder.split(os.sep)[-2] + '.tif')
channel2_file = os.path.join(folder, 'r_' + folder.split(os.sep)[-2] + '.tif')
dapi_file = os.path.join(folder, 'b_' + folder.split(os.sep)[-2] + '.tif')

img_channel1 = np.array(read_image_stack(channel1_file), dtype=float)
img_channel2 = np.array(read_image_stack(channel2_file), dtype=float)
img_dapi = np.array(read_image_stack(dapi_file), dtype=float)

# spot detection

we will now detect spots in the images.
to do that, enter pixelsize (in the form [x,y,x]) and expected size (fwhm) of spots (some form and unit as pixelsize)
you can also select intensity thresholds for the two channels
* Orange: channel1
* Red: channel2
this will display 2 images:
* xy-maxprojection with spots labeled
* xz-maxprojection with spots labeled

In [ ]:
pixelsize = [40, 40, 300]
expected_size = [250, 250, 500]

threshold_channel1 = 25
threshold_channel2 = 15

z_resample = pixelsize[2] / pixelsize[0]
print('z-resampling: ' + str(z_resample))

sigmas = np.array(expected_size, dtype=float) / np.array(pixelsize, dtype=float) / (2. * np.sqrt(2. * np.log(2.)))
sigmas[sigmas < 1] = 1.0
print('sigmas: ' + str(sigmas))

img_rgb = get_rgb_projected(img_channel1, img_channel2, img_dapi)
img_rgb_xz = get_rgb_projected(img_channel1, img_channel2, img_dapi, axis=1)
img_rgb_xz = zoom(img_rgb_xz, (1, z_resample, 1), order=1)

log1 = - gaussian_laplace(img_channel1, sigmas)
pks1 = peak_local_max(log1, threshold_abs=threshold_channel1)
log2 = - gaussian_laplace(img_channel2, sigmas)
pks2 = peak_local_max(log2, threshold_abs=threshold_channel2)

plt.imshow(img_rgb)

for i in range(len(pks1)):
    d = pks1[i]
    c = plt.Circle((d[1], d[0]), 8, color='lightsalmon', linewidth=1, fill=False)
    plt.text(d[1] - 10, d[0] - 10, str(i), color='white')
    plt.gca().add_patch(c)
    plt.draw()

for i in range(len(pks2)):
    d = pks2[i]
    c = plt.Circle((d[1], d[0]), 8, color='palegreen', linewidth=1, fill=False)
    plt.text(d[1] - 10, d[0] - 10, str(i), color='palegreen')
    plt.gca().add_patch(c)
    plt.draw()    

plt.figure()
plt.imshow(img_rgb_xz)

for i in range(len(pks1)):
    d = pks1[i]
    c = plt.Circle((d[2] * z_resample, d[0]), 8, color='lightsalmon', linewidth=1, fill=False)
    plt.text(d[2] * z_resample - 10, d[0] - 10, str(i), color='white')
    plt.gca().add_patch(c)
    plt.draw()

for i in range(len(pks2)):
    d = pks2[i]
    c = plt.Circle((d[2] * z_resample, d[0]), 8, color='palegreen', linewidth=1, fill=False)
    plt.text(d[2] * z_resample - 10, d[0] - 10, str(i), color='palegreen')
    plt.gca().add_patch(c)
    plt.draw()    


# remove detections

in the cell below, you can remove detected spots by hand.
to do that, just add the indices of the spots in the images displayed above (orange is displayed as red, red as green)
in the respective list

*NOTE:* you have to enecute the code below even if you do not want to remove spots - in that case, just leave the lists empty

In [ ]:
points_to_remove_channel1 = []
points_to_remove_channel2 = [2,3]

pks1i = pks1[[i for i in range(pks1.shape[0]) if i not in points_to_remove_channel1]]
pks2i = pks2[[i for i in range(pks2.shape[0]) if i not in points_to_remove_channel2]]

plt.imshow(img_rgb)

for i in range(len(pks1i)):
    d = pks1i[i]
    c = plt.Circle((d[1], d[0]), 8, color='lightsalmon', linewidth=1, fill=False)
    plt.text(d[1] - 10, d[0] - 10, str(i), color='white')
    plt.gca().add_patch(c)
    plt.draw()

for i in range(len(pks2i)):
    d = pks2i[i]
    c = plt.Circle((d[1], d[0]), 8, color='palegreen', linewidth=1, fill=False)
    plt.text(d[1] - 10, d[0] - 10, str(i), color='palegreen')
    plt.gca().add_patch(c)
    plt.draw()    

plt.figure()
plt.imshow(img_rgb_xz)

for i in range(len(pks1i)):
    d = pks1i[i]
    c = plt.Circle((d[2] * z_resample, d[0]), 8, color='lightsalmon', linewidth=1, fill=False)
    plt.text(d[2] * z_resample - 10, d[0] - 10, str(i), color='white')
    plt.gca().add_patch(c)
    plt.draw()

for i in range(len(pks2i)):
    d = pks2i[i]
    c = plt.Circle((d[2] * z_resample, d[0]), 8, color='palegreen', linewidth=1, fill=False)
    plt.text(d[2] * z_resample - 10, d[0] - 10, str(i), color='palegreen')
    plt.gca().add_patch(c)
    plt.draw()    

# find spot pairs and display them
we will now determine spot pairs via linear assignment and display them

we have 2 manual intervention methods here:

* by adding a pair of spots (as a list [idxA, idxB]) to the list lines_to_ignore, we can try to find a matching without a link between spots A and B (use indices from the output of spot filtering)

* finally, we can simply discard a link by adding its index (from the output below) to lines_to_remove

In [ ]:
lines_to_ignore = []
lines_to_remove = []


cij = distance_mat(pks1i, pks2i, lines_to_ignore, np.array(pixelsize, dtype=float))
row, col = linear_sum_assignment(cij)

plt.imshow(img_rgb)

for i in row:
    if i in lines_to_remove:
        continue
    if not i in range(pks1i.shape[0]):
        continue
    if not col[i] in range(pks2i.shape[0]):
        continue
    p1 = pks1i[i]
    p2 = pks2i[col[i]]
    plt.text(p1[1] - 10, p1[0] - 10, str(i), color='white')
    plt.plot([p1[1], p2[1]], [p1[0], p2[0]], '-', color='white', lw=2)
    plt.draw()
    
plt.figure()
plt.imshow(img_rgb_xz)

for i in row:
    if i in lines_to_remove:
        continue
    if not i in range(pks1i.shape[0]):
        continue
    if not col[i] in range(pks2i.shape[0]):
        continue
    p1 = pks1i[i]
    p2 = pks2i[col[i]]
    plt.text(p1[2] * z_resample - 10, p1[0] - 10, str(i), color='white')
    plt.plot([p1[2] * z_resample, p2[2] * z_resample], [p1[0], p2[0]], '-', color='white', lw=2)
    plt.draw()

# save results
the code below will save the results to a .csv file in the folder that was processed

In [ ]:
res = pd.DataFrame()

xch1 = []
xch2 = []
ych1 = []
ych2 = []
zch1 = []
zch2 = []

for i in row:
    if i in lines_to_remove:
        continue
    if not i in range(pks1i.shape[0]):
        continue
    if not col[i] in range(pks2i.shape[0]):
        continue
    p1 = pks1i[i] * np.array(pixelsize, dtype=float)
    p2 = pks2i[col[i]] * np.array(pixelsize, dtype=float)
    
    xch1.append(p1[1])
    xch2.append(p2[1])
    ych1.append(p1[0])
    ych2.append(p2[0])
    zch1.append(p1[2])
    zch2.append(p2[2])
    
res['xch1'] = xch1
res['xch2'] = xch2
res['ych1'] = ych1
res['ych2'] = ych2
res['zch1'] = zch1
res['zch2'] = zch2

res['distance'] = np.sqrt((res.xch1 - res.xch2)**2 + (res.ych1 - res.ych2)**2 + (res.zch1 - res.zch2)**2)

res.to_csv(os.path.join(folder, 'result.csv'))
res